In [ ]:
%%capture
!pip install --upgrade transformers
!pip install torchaudio
!pip install jiwer
!pip install accelerate -U

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
from datasets import load_dataset

common_voice_train = load_dataset(
    "mozilla-foundation/common_voice_16_0",
    "hy-AM",
    split="train+validation",
)

common_voice_test = load_dataset(
    "mozilla-foundation/common_voice_16_0",
    "hy-AM",
    split="test",
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

common_voice_16_0.py:   0%|          | 0.00/8.17k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/77.3k [00:00<?, ?B/s]

...


Reading metadata...: 2692it [00:00, 42077.62it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 6215it [00:00, 44810.25it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 694it [00:00, 46473.91it/s]


In [ ]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])


In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)


,sentence,variant
0,Վահանների ուրվագծերը աչքի են ընկնում մեծ կայունությամբ երկարատև ժամանակահատվածի մեջ։,
1,"Բարձրացել է գյուղատնտեսության կուլտուրաների բերքատվությունը, ավելացել ապրանքային արտադրությունը։",
2,Գրվածքը վերաբերում էր մահվան և անմահության փիլիսոփայական բացատրության:,
3,"Մեջքային մակերեսին առկա են բազմազան, հաճախ՝ սուր, կատարանման ու փշանման թեփուկներ։",
4,Նա ևս ուներ նկարչի և քանդակագործի ձիրք։,
5,Արջը կոկորդիլոսին հրավիրում է մարտի և փրկում արևը։,
6,Կան նաև անանուն ժամանակագրություններ։,
7,"Քանի որ ցողունը առանցքային օրգան է, այն ունի շառավղային կառուցվածք։",
8,"- Փող ունենամ,- ասաց,- մի լավ գլխարկ եմ տեսել, կգնամ կառնեմ, կծածկեմ:",
9,Կոշութի անվան մրցանակի կրկնակի դափնեկիր։,


In [ ]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\»\«]'

def remove_special_characters(batch):
    # remove special characters
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()

    return batch

common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)


Map:   0%|          | 0/5993 [00:00<?, ? examples/s]

Map:   0%|          | 0/2692 [00:00<?, ? examples/s]

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))


,sentence,variant
0,պորֆիրինի որոշ ածանցյալներ մեծ հետաքրքրություն են առաջացնում ուռուցքաբանական հիվանդությունների թերապիայի տեսանկյունից։,
1,ըստ հեռուստասերիալի սյուժեի նա ապրում է երկու կյանքով։,
2,շինությունը զարդարված է փայտե բազմաթիվ նախշերով։,
3,նրանց կարելի է սովորեցնել տարբեր բառեր ասել։,
4,կլաուզևիցի ձևակերպմամբ պատերազմը քաղաքականության շարունակությունն է այլ բռնի միջոցներով։,
5,այսօր երգի բառերն ու երաժշտությունը պատկանում են հանրության սեփականությանը։,
6,հնում այսպես էին կոչում անտիկ պոեզիայի լավագույն ստեղծագործությունների ժողովածուները։,
7,եղած է կերպարուեստի սիրահար եւ հաւաքիչ։,
8,իրիսը հաճախակի ներկայացվում է ձեռքին գավաթ և լայն բացված թևերով թռչելիս։,
9,եվ միայն այն պատճառով որ նա կիսով չափ սպիտակ էր ծնվել։,


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)


Map:   0%|          | 0/5993 [00:00<?, ? examples/s]

Map:   0%|          | 0/2692 [00:00<?, ? examples/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict


{' ': 0,
 '(': 1,
 ')': 2,
 '`': 3,
 '´': 4,
 '՚': 5,
 '՛': 6,
 '՜': 7,
 '՝': 8,
 '՞': 9,
 'ա': 10,
 'բ': 11,
 'գ': 12,
 'դ': 13,
 'ե': 14,
 'զ': 15,
 'է': 16,
 'ը': 17,
 'թ': 18,
 'ժ': 19,
 'ի': 20,
 'լ': 21,
 'խ': 22,
 'ծ': 23,
 'կ': 24,
 'հ': 25,
 'ձ': 26,
 'ղ': 27,
 'ճ': 28,
 'մ': 29,
 'յ': 30,
 'ն': 31,
 'շ': 32,
 'ո': 33,
 'չ': 34,
 'պ': 35,
 'ջ': 36,
 'ռ': 37,
 'ս': 38,
 'վ': 39,
 'տ': 40,
 'ր': 41,
 'ց': 42,
 'ւ': 43,
 'փ': 44,
 'ք': 45,
 'օ': 46,
 'ֆ': 47,
 'և': 48,
 '։': 49,
 '֊': 50,
 '’': 51,
 '…': 52}

In [ ]:
def remove_latin_characters(batch):
    batch["sentence"] = re.sub(r'[a-z]+', '', batch["sentence"])
    return batch

# remove latin characters
common_voice_train = common_voice_train.map(remove_latin_characters)
common_voice_test = common_voice_test.map(remove_latin_characters)

# extract unique characters again
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict


Map:   0%|          | 0/5993 [00:00<?, ? examples/s]

Map:   0%|          | 0/2692 [00:00<?, ? examples/s]

Map:   0%|          | 0/5993 [00:00<?, ? examples/s]

Map:   0%|          | 0/2692 [00:00<?, ? examples/s]

{' ': 0,
 '(': 1,
 ')': 2,
 '`': 3,
 '´': 4,
 '՚': 5,
 '՛': 6,
 '՜': 7,
 '՝': 8,
 '՞': 9,
 'ա': 10,
 'բ': 11,
 'գ': 12,
 'դ': 13,
 'ե': 14,
 'զ': 15,
 'է': 16,
 'ը': 17,
 'թ': 18,
 'ժ': 19,
 'ի': 20,
 'լ': 21,
 'խ': 22,
 'ծ': 23,
 'կ': 24,
 'հ': 25,
 'ձ': 26,
 'ղ': 27,
 'ճ': 28,
 'մ': 29,
 'յ': 30,
 'ն': 31,
 'շ': 32,
 'ո': 33,
 'չ': 34,
 'պ': 35,
 'ջ': 36,
 'ռ': 37,
 'ս': 38,
 'վ': 39,
 'տ': 40,
 'ր': 41,
 'ց': 42,
 'ւ': 43,
 'փ': 44,
 'ք': 45,
 'օ': 46,
 'ֆ': 47,
 'և': 48,
 '։': 49,
 '֊': 50,
 '’': 51,
 '…': 52}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]


In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)


55

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)


In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")


In [ ]:
repo_name = "w2v-bert-2.0-armenian-master"

In [ ]:
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/SergeyAdamyan/w2v-bert-2.0-armenian-master/commit/dfb545e50cc57cefdc0de08fbabb00121873e68e', commit_message='Upload tokenizer', commit_description='', oid='dfb545e50cc57cefdc0de08fbabb00121873e68e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/SergeyAdamyan/w2v-bert-2.0-armenian-master', endpoint='https://huggingface.co', repo_type='model', repo_id='SergeyAdamyan/w2v-bert-2.0-armenian-master'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import SeamlessM4TFeatureExtractor

feature_extractor = SeamlessM4TFeatureExtractor.from_pretrained("facebook/w2v-bert-2.0")


preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

In [ ]:
from transformers import Wav2Vec2BertProcessor

processor = Wav2Vec2BertProcessor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
processor.push_to_hub(repo_name)

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SergeyAdamyan/w2v-bert-2.0-armenian-master/commit/09a59f31750e4578b16a017751a002a2dd75eb81', commit_message='Upload processor', commit_description='', oid='09a59f31750e4578b16a017751a002a2dd75eb81', pr_url=None, repo_url=RepoUrl('https://huggingface.co/SergeyAdamyan/w2v-bert-2.0-armenian-master', endpoint='https://huggingface.co', repo_type='model', repo_id='SergeyAdamyan/w2v-bert-2.0-armenian-master'), pr_revision=None, pr_num=None)

In [ ]:
common_voice_train[0]["path"]


'/root/.cache/huggingface/datasets/downloads/extracted/451e713ec4680484ed239ecfb9f6679dcb6922473fb7d5d5e7d720ac8a80fb9c/hy-AM_train_0/common_voice_hy-AM_39459169.mp3'

In [ ]:
common_voice_train[0]["audio"]


{'path': '/root/.cache/huggingface/datasets/downloads/extracted/451e713ec4680484ed239ecfb9f6679dcb6922473fb7d5d5e7d720ac8a80fb9c/hy-AM_train_0/common_voice_hy-AM_39459169.mp3',
 'array': array([0., 0., 0., ..., 0., 0., 0.]),
 'sampling_rate': 48000}

In [ ]:
from datasets import Audio


In [ ]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))


In [ ]:
common_voice_train[0]["audio"]


{'path': '/root/.cache/huggingface/datasets/downloads/extracted/451e713ec4680484ed239ecfb9f6679dcb6922473fb7d5d5e7d720ac8a80fb9c/hy-AM_train_0/common_voice_hy-AM_39459169.mp3',
 'array': array([0., 0., 0., ..., 0., 0., 0.]),
 'sampling_rate': 16000}

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)


սպորտի վարպետի թեկնածու է մարզական պարահանդեսային պարերի գծով։


In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])


Target text: էլիսոնը նաև երգչուհի մոդել և ֆոտոմոդել է։
Input array shape: (85248,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_features"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["input_length"] = len(batch["input_features"])

    batch["labels"] = processor(text=batch["sentence"]).input_ids
    return batch


In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)


Map:   0%|          | 0/5993 [00:00<?, ? examples/s]

Map:   0%|          | 0/2692 [00:00<?, ? examples/s]

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:

    processor: Wav2Vec2BertProcessor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        labels_batch = self.processor.pad(
            labels=label_features,
            padding=self.padding,
            return_tensors="pt",
        )
        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
import evaluate

wer_metric = evaluate.load("wer")


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [ ]:
from transformers import Wav2Vec2BertForCTC

model = Wav2Vec2BertForCTC.from_pretrained(
    "facebook/w2v-bert-2.0",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.0,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    add_adapter=True,
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

Some weights of Wav2Vec2BertForCTC were not initialized from the model checkpoint at facebook/w2v-bert-2.0 and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2_bert.adapter.layers.0.ffn.intermediate_dense.bias', 'wav2vec2_bert.adapter.layers.0.ffn.intermediate_dense.weight', 'wav2vec2_bert.adapter.layers.0.ffn.output_dense.bias', 'wav2vec2_bert.adapter.layers.0.ffn.output_dense.weight', 'wav2vec2_bert.adapter.layers.0.ffn_layer_norm.bias', 'wav2vec2_bert.adapter.layers.0.ffn_layer_norm.weight', 'wav2vec2_bert.adapter.layers.0.residual_conv.bias', 'wav2vec2_bert.adapter.layers.0.residual_conv.weight', 'wav2vec2_bert.adapter.layers.0.residual_layer_norm.bias', 'wav2vec2_bert.adapter.layers.0.residual_layer_norm.weight', 'wav2vec2_bert.adapter.layers.0.self_attn.linear_k.bias', 'wav2vec2_bert.adapter.layers.0.self_attn.linear_k.weight', 'wav2vec2_bert.adapter.layers.0.self_attn.linear_out.bias', 'wav2vec2_bert.adapter.layers.0.self_attn.linear_out.weight', 'wav2vec2_ber

In [ ]:
from transformers import TrainingArguments
print(TrainingArguments.__module__)


transformers.training_args


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  num_train_epochs=10,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=600,
  eval_steps=300,
  logging_steps=300,
  learning_rate=5e-5,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
  eval_strategy="steps",
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor,
)


/tmp/ipython-input-38-1973840082.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
from transformers import AutoModelForCTC, Wav2Vec2BertProcessor

model = AutoModelForCTC.from_pretrained("ylacombe/w2v-bert-2.0-mongolian-colab-CV16.0")
processor = Wav2Vec2BertProcessor.from_pretrained("ylacombe/w2v-bert-2.0-mongolian-colab-CV16.0")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.42G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json:   0%|          | 0.00/870 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sergeyadamyan (sergeyadamyan-yerevan-state-university-ysu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Wer
300,2.033100,0.259488,0.323882
600,0.200900,0.224702,0.232321
900,0.123700,0.188506,0.217308
1200,0.071800,0.176477,0.182644
1500,0.039000,0.186576,0.170356
1800,0.016700,0.194367,0.172125


TrainOutput(global_step=1880, training_loss=0.39702039418068336, metrics={'train_runtime': 12667.3052, 'train_samples_per_second': 4.731, 'train_steps_per_second': 0.148, 'total_flos': 9.47387596230927e+18, 'train_loss': 0.39702039418068336, 'epoch': 10.0})

In [ ]:
from transformers import Wav2Vec2BertForCTC, Wav2Vec2BertProcessor

repo_name = "SergeyAdamyan/w2v-bert-2.0-armenian-master"
# Load processor and model
processor = Wav2Vec2BertProcessor.from_pretrained(repo_name)
model = Wav2Vec2BertForCTC.from_pretrained(repo_name).to("cuda")

preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.42G [00:00<?, ?B/s]

In [ ]:
sample = common_voice_test[0]
input_features = torch.tensor(sample["input_features"]).to("cuda").unsqueeze(0)

with torch.no_grad():
    logits = model(input_features).logits

pred_ids = torch.argmax(logits, dim=-1)[0]


In [ ]:
print(processor.decode(pred_ids))
print(processor.decode(sample["labels"]).lower())


միջպետական հարաբերություններում կարևոր դեր էին խաղում քաղաքական ամուսնությունները։
միջպետական հարաբերություներում կարևոր դեր էին խաղում քաղաքական ամուսնություները։


In [ ]:
for i in range(100):
    sample = common_voice_test[i]

    input_features = torch.tensor(sample["input_features"]).to("cuda").unsqueeze(0)

    with torch.no_grad():
        logits = model(input_features).logits

    pred_ids = torch.argmax(logits, dim=-1)[0]

    predicted_text = processor.decode(pred_ids)
    actual_text = processor.decode(sample["labels"]).lower()

    print(f"Sample {i+1}")
    print("Predicted:", predicted_text)
    print("Actual:   ", actual_text)
    print("-" * 50)

Sample 1
Predicted: միջպետական հարաբերություններում կարևոր դեր էին խաղում քաղաքական ամուսնությունները։
Actual:    միջպետական հարաբերություներում կարևոր դեր էին խաղում քաղաքական ամուսնություները։
--------------------------------------------------
Sample 2
Predicted: այս դեպքում նրանք ձեռք են բերում նույն կենսացենոզում համագոյակցելու ունակություն։
Actual:    այս դեպքում նրանք ձեռք են բերում նույն կենսացենոզում համագոյակցելու ունակություն։
--------------------------------------------------
Sample 3
Predicted: նրա երգերը դեռևս հայտնի են եգիպտացիների շրջանում սակայն ռադիոյով չի հնչել։
Actual:    նրա երգերը դեռևս հայտնի էին եգիպտացիների շրջանում սակայն ռադիոյով չի հնչել։
--------------------------------------------------
Sample 4
Predicted: խորհուրդ է տրվում կիրառել մեթոդիկան հոգեթերապիայի կամ կարճատև հոգեվելուծության նախապատրաստման համար։
Actual:    խորհուրդ է տրվում կիրառել մեթոդիկան հոգեթերապիայի կամ կարճատև հոգեվերլուծության նախապատրաստման համար։
-----------------------------------------